In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
data = pd.read_csv('data/p4kreviews.csv', encoding='latin1')

In [7]:
data.head()

,Unnamed: 0,album,artist,best,date,genre,review,score
0,1,A.M./Being There,Wilco,1,December 6 2017,Rock,Best new reissue 1 / 2 Albums Newly reissued a...,7.0
1,2,No Shame,Hopsin,0,December 6 2017,Rap,"On his corrosive fifth album, the rapper takes...",3.5
2,3,Material Control,Glassjaw,0,December 6 2017,Rock,"On their first album in 15 years, the Long Isl...",6.6
3,4,Weighing of the Heart,Nabihah Iqbal,0,December 6 2017,Pop/R&B,"On her debut LP, British producer Nabihah Iqba...",7.7
4,5,The Visitor,Neil Young / Promise of the Real,0,December 5 2017,Rock,"While still pointedly political, Neil Youngs ...",6.7


In [16]:
grouped_by_artist = data.groupby(['artist']).agg(['count'])
grouped_by_artist

,Unnamed: 0,album,best,date,genre,review,score
,count,count,count,count,count,count,count
artist,,,,,,,
!!!,7,7,7,7,7,7,7
!!! / Out Hud,1,1,1,1,1,1,1
(Sandy) Alex G,1,1,1,1,1,1,1
+/-,6,6,6,6,6,6,6
...And You Will Know Us by the Trail of Dead,10,10,10,10,10,10,10
"1,2,3",1,1,1,1,1,1,1
10 in the Swear Jar,1,1,1,1,1,1,1
11,1,1,1,1,1,1,1


In [15]:
grouped_by_artist.sort_values(by=['album'])

ValueError: Cannot sort by column album in a multi-index you need to explicitly provide all the levels